In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
import numpy as np
import pandas as pd
import random

In [2]:
meta=[]
sequence=[]

In [3]:
seq = ('LongInternalExon_SNM.txt')

In [4]:
for seq_record in SeqIO.parse(seq, "fasta"):
    meta.append(str(seq_record.id))
    sequence.append(str(seq_record.seq))

In [5]:
Fasta = pd.DataFrame(data ={'Meta':meta,'Sequence':sequence})

In [6]:
Data = pd.read_csv("LongInternalExon_SNM.csv")

In [7]:
Result = pd.merge(Data, Fasta, left_index=True, right_index=True, how='outer')

In [8]:
Result.drop(["Meta"], axis=1, inplace=True)

In [9]:
Result

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,CumSum,m6AStart,m6AEnd,Probabilty,Label,Sequence
0,Satb1,chr17,-,51736186,51832672,51739897,51809253,51736186,51740413,4227,6243,51739922,51739923,0.936465,1,TCATGTTTTAAACATTAATTCCAAACTATTTTTATTTAGCATATGG...
1,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,6755,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...
2,Jag1,chr2,-,137081455,137116644,137083023,137116257,137081455,137083481,2026,5612,137083096,137083097,0.930130,1,CATGTTTTCATACaaaaaatttaataaatattacttttcaaaattt...
3,Six4,chr12,-,73099608,73113425,73103441,73113184,73108660,73109349,689,1772,73109072,73109073,0.928997,1,TATGTATAAATATAAAGTTTATTTAGCATAGTGTTTAGAAAAATAA...
4,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,4974,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Whsc1l1,chr8,+,25640283,25719667,25640620,25714400,25713357,25713559,202,4436,25713496,25713497,0.401583,1,TGTGTGAAGATTACTTATGCATCCTAAAGTTGCTATGGAAGTAAGC...
1456,Zfp52,chr17,+,21555045,21562066,21555045,21562066,21560048,21562066,2018,2175,21561954,21561955,0.400855,1,ATGGCCAATTCTCCAGTAAACACATCCCAGGTCAGTGGTCATGTCC...
1457,Nfe2l1,chr11,-,96817413,96829968,96819175,96827707,96827197,96828223,1026,1176,96827489,96827490,0.400468,1,GCACTTAAGCTGAATCATCCATTTTATTTGATTTTATTTTGTCATG...
1458,Sfrp1,chr8,+,23411501,23449632,23411784,23446488,23446165,23449632,3467,4372,23446405,23446406,0.400438,1,ATTCGAGCTGCAGCGTCGCAAGTAGGTGCAGCAGCCCGCAGCCCGC...


In [10]:
Result["GeneLength"] = Result["txEnd"] - Result["txStart"]

In [11]:
Data = Result

In [12]:
Positive = Data[Data["strand"]=="+"]
Positive = Positive.reset_index(drop = True)
Negative = Data[Data["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [13]:
Sequence = Negative["Sequence"].tolist()

In [14]:
New = []
for i in range(len(Sequence)):
    input_sequence = Sequence[i][::-1]
    input_sequence = Seq(input_sequence).complement()
    input_sequence = str(input_sequence)
    New.append(input_sequence)

New = np.array(New)
New = pd.DataFrame({"Sequence":New})

In [15]:
Negative["Sequence"] = 0
Negative["Sequence"] = New["Sequence"]

In [16]:
Data = Positive.append(Negative)
Data = Data.reset_index(drop = True)

In [17]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,CumSum,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength
0,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,6755,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071
1,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,4974,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551
2,Ppp1r15b,chr1,+,133131142,133139783,133131746,133136887,133136662,133139783,3121,5594,133136855,133136856,0.909691,1,GAGTAAGGAAGCCGACAGGGACCACAGACGCGGCCAATCAGAGGCC...,8641
3,Fzd4,chr7,+,89404365,89410110,89404686,89408360,89407031,89410110,3079,3685,89407633,89407634,0.908575,1,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,5745
4,Pard6b,chr2,+,168081003,168101203,168081293,168099210,168098383,168101203,2820,3399,168099166,168099167,0.906320,1,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,20200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Mapk1ip1,chr7,-,138835832,138846260,138835907,138836699,138835832,138837030,1198,1502,138836056,138836057,0.403412,1,GCTGTCTTGAGGTTGGACCTACGTAGCCGCTGCGGTCCAAGGAGAG...,10428
1456,Lrrn3,chr12,-,41451672,41485751,41452192,41454316,41451672,41454671,2999,3180,41452631,41452632,0.403062,1,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,34079
1457,Lrp5,chr19,-,3584827,3686556,3585029,3686442,3659243,3659640,397,599,3659463,3659464,0.401945,1,GGGCCGCGGCGCCCGAGGCGGGAGCAAGAGGCGCCGGGAGCCGCGA...,101729
1458,Ttll11,chr2,-,35751240,35979624,35752232,35979624,35902569,35903145,576,1260,35902739,35902740,0.401592,1,ATGCGGCGCAGCAGCCCGGAGAAGAAGCCGGAGGCTGAGTGGGAGG...,228384


In [18]:
Positive = Data[Data["strand"]=="+"]
Positive = Positive.reset_index(drop = True)
Negative = Data[Data["strand"]=="-"]
Negative = Negative.reset_index(drop = True)

In [19]:
Positive["POS"] = Positive["m6AStart"] - Positive["txStart"]
Negative["POS"] = Negative["txEnd"] - Negative["m6AEnd"]

In [20]:
Data = Positive.append(Negative)
Data = Data.reset_index(drop = True)

In [21]:
for i in range(len(Data)):
    Data.loc[i,("m6ASequence")] = Data.loc[i,("Sequence")][(Data.loc[i,("POS")]-2):(Data.loc[i,("POS")]+3)]

In [22]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,CumSum,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength,POS,m6ASequence
0,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,6755,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071,74096,GGACT
1,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,4974,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551,448476,GAACT
2,Ppp1r15b,chr1,+,133131142,133139783,133131746,133136887,133136662,133139783,3121,5594,133136855,133136856,0.909691,1,GAGTAAGGAAGCCGACAGGGACCACAGACGCGGCCAATCAGAGGCC...,8641,5713,GGACT
3,Fzd4,chr7,+,89404365,89410110,89404686,89408360,89407031,89410110,3079,3685,89407633,89407634,0.908575,1,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,5745,3268,GGACT
4,Pard6b,chr2,+,168081003,168101203,168081293,168099210,168098383,168101203,2820,3399,168099166,168099167,0.906320,1,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,20200,18163,GGACC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Mapk1ip1,chr7,-,138835832,138846260,138835907,138836699,138835832,138837030,1198,1502,138836056,138836057,0.403412,1,GCTGTCTTGAGGTTGGACCTACGTAGCCGCTGCGGTCCAAGGAGAG...,10428,10203,taacc
1456,Lrrn3,chr12,-,41451672,41485751,41452192,41454316,41451672,41454671,2999,3180,41452631,41452632,0.403062,1,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,34079,33119,AGACT
1457,Lrp5,chr19,-,3584827,3686556,3585029,3686442,3659243,3659640,397,599,3659463,3659464,0.401945,1,GGGCCGCGGCGCCCGAGGCGGGAGCAAGAGGCGCCGGGAGCCGCGA...,101729,27092,AGACC
1458,Ttll11,chr2,-,35751240,35979624,35752232,35979624,35902569,35903145,576,1260,35902739,35902740,0.401592,1,ATGCGGCGCAGCAGCCCGGAGAAGAAGCCGGAGGCTGAGTGGGAGG...,228384,76884,GGACT


In [23]:
Data.drop(["m6ASequence"], axis=1, inplace=True)

In [24]:
for i in range(len(Data)):
    Data.loc[i,("Up")] = Data.loc[i,("Sequence")][:(Data.loc[i,("POS")]-50)]
    Data.loc[i,("Mid")] = Data.loc[i,("Sequence")][(Data.loc[i,("POS")]-50):(Data.loc[i,("POS")]+51)]
    Data.loc[i,("Down")] = Data.loc[i,("Sequence")][(Data.loc[i,("POS")]+51):]

In [25]:
Data

,name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonStart,exonEnd,ExonLength,...,m6AStart,m6AEnd,Probabilty,Label,Sequence,GeneLength,POS,Up,Mid,Down
0,Setd5,chr6,+,113077364,113153435,113105050,113151470,113150861,113153435,2574,...,113151460,113151461,0.930645,1,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,76071,74096,AGTGGCGTTTCCGCTCGGGCAGCGGGCTGAGTGAgctgccgccgct...,CAGTACCGACTCCAGCCACTGCAAGGGTCAGGAGTCAAGACTCAGA...,CTGCTTCCTTCCAGCTGCCTCTGGAATCAAGGCCGAGCATATCGCT...
1,Zfpm2,chr15,+,40655041,41104592,40655543,41103576,41101084,41104592,3508,...,41103517,41103518,0.920117,1,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,449551,448476,GCGCGGCGCGGCCCGGAGCGGCAGCGGCGGCGCCTGAGTCTCTGCC...,CAGTGGAAAATACTGCCGGCTATGCGATATTCAGTTCAATAATCTT...,TCAAATGAACTAACTGAGTTACTAAAGAAAGCAGTCACCTTTGGTA...
2,Ppp1r15b,chr1,+,133131142,133139783,133131746,133136887,133136662,133139783,3121,...,133136855,133136856,0.909691,1,GAGTAAGGAAGCCGACAGGGACCACAGACGCGGCCAATCAGAGGCC...,8641,5713,GAGTAAGGAAGCCGACAGGGACCACAGACGCGGCCAATCAGAGGCC...,CTTTGAGCACAGAGAAAAAATGTTTAATAGACTGAGGATCGAGTCA...,CACTCTGTCTCTTACTTGAGAGTTTCCCTTAAAAACAAACACTGGC...
3,Fzd4,chr7,+,89404365,89410110,89404686,89408360,89407031,89410110,3079,...,89407633,89407634,0.908575,1,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,5745,3268,AGCGCTGGGGCGGTGAGAACAGCGCGGCGTAGAGTGCAGGCGGGCT...,ATATCCTGTGATTTTGAAGAGGCGGCAGAGCCCGTTCTCATCCAAG...,AATGGCCAGCTCCATTTGGTGGGTTATTCTGACACTCACTTGGTTT...
4,Pard6b,chr2,+,168081003,168101203,168081293,168099210,168098383,168101203,2820,...,168099166,168099167,0.906320,1,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,20200,18163,AGGGAGGCTGCGGCGGCTGTAGGGCTGTGCTCCGCCCTCTCCTCAc...,GGTGCCTGTGCCCGGCAGCCTGGACACAGAGCTGGAAAGCCGGGCT...,GGTTGTTTTCTGTTTGAGGGTGTAAGGAACAATGTTGGTGTGCACA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,Mapk1ip1,chr7,-,138835832,138846260,138835907,138836699,138835832,138837030,1198,...,138836056,138836057,0.403412,1,GCTGTCTTGAGGTTGGACCTACGTAGCCGCTGCGGTCCAAGGAGAG...,10428,10203,GCTGTCTTGAGGTTGGACCTACGTAGCCGCTGCGGTCCAAGGAGAG...,acaacaaatccatcaagaggagacgatccaagaagaaaagcaagcg...,caacaaggacacaataccactgaccccaaaatgatgctgctatgtt...
1456,Lrrn3,chr12,-,41451672,41485751,41452192,41454316,41451672,41454671,2999,...,41452631,41452632,0.403062,1,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,34079,33119,agagagaCACCATTGCTTGCATGTGTGCATATCTATAGTATGTAAT...,TAAAATCCTCAAATCCAGCGTTAAGTGGACAGCCTTTGTCAAGACT...,ATCTTACTCATCTGAAACCATCTACCGAGTACAAAATTTGTATTGA...
1457,Lrp5,chr19,-,3584827,3686556,3585029,3686442,3659243,3659640,397,...,3659463,3659464,0.401945,1,GGGCCGCGGCGCCCGAGGCGGGAGCAAGAGGCGCCGGGAGCCGCGA...,101729,27092,GGGCCGCGGCGCCCGAGGCGGGAGCAAGAGGCGCCGGGAGCCGCGA...,CCAAGGGTGCTGTGTACTGGACAGATGTGAGCGAGGAGGCCATCAA...,CTCGTGTCACCTGATGGCCTGGCCTGTGACTGGGTTGGCAAGAAGC...
1458,Ttll11,chr2,-,35751240,35979624,35752232,35979624,35902569,35903145,576,...,35902739,35902740,0.401592,1,ATGCGGCGCAGCAGCCCGGAGAAGAAGCCGGAGGCTGAGTGGGAGG...,228384,76884,ATGCGGCGCAGCAGCCCGGAGAAGAAGCCGGAGGCTGAGTGGGAGG...,ACAGCGGCAAGTTTGTCCACTCTGACAGCGCCAGCACGGGCAGCAA...,AAGGTCTGGTCTGATATCATCTCCTTAGTTATTAAGACTGTCATCG...


In [26]:
Data.to_csv("Fasta_SNM_LongInternalExon.csv", index=0)